In [93]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification 
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm

import sys
sys.path.append("..")
from src.models import EModel, AASIST, Wav2Vec2Facebook
from src.datamodules import AASIST2DataModule, AASISTCenterLossDataset

In [94]:
# model_name_or_path = "facebook/wav2vec2-base"
# feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
# sampling_rate = feature_extractor.sampling_rate

path = "/home/work/joono/joono/joono/DV_DV.Deep/f23cmc1s/checkpoints/best-checkpoint.ckpt"
# model = Wav2Vec2Facebook.load_from_checkpoint(path, args={})
model = AASIST.load_from_checkpoint(path, args={})

In [95]:
# Collate 함수 정의
def collate_fn(batch):
    # signals = zip(*batch)
    signals = batch
    max_length = max([signal.size(0) for signal in signals])
    padded_signals = torch.zeros(len(signals), max_length)
    for i, signal in enumerate(signals):
        padded_signals[i, :signal.size(0)] = signal
    # labels = torch.tensor(labels)
    return padded_signals

In [96]:
test_df = pd.read_csv('../dataset/test.csv', index_col=None)
test_df['path'] = '../dataset/' + test_df['path'].str[1:]
# test_dataset = TestDataset(test_df)
test_dataset = AASISTCenterLossDataset(test_df, train_mode=False)
# test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=24, batch_size=32, collate_fn=collate_fn)

In [97]:
def inference(model, test_loader, device):
    model.to(device)
    model = model.eval()
    predictions, categories = [], []
    with torch.no_grad():
        # for inputs, labels in tqdm(test_loader):
        for inputs in tqdm(test_loader):
            
            inputs = inputs.to(device)
            # labels = labels.to(device)
            
            probs, _, category = model(inputs)

            categories+= category.cpu().detach().numpy().tolist()
            predictions += probs.cpu().detach().numpy().tolist()
            
    return predictions, categories

In [98]:
preds, categories = inference(model=model, test_loader=test_loader, device='cuda:1')
# preds = model.inference(test_loader=test_loader)

  0%|          | 0/1563 [00:00<?, ?it/s]

100%|██████████| 1563/1563 [03:03<00:00,  8.51it/s]


In [104]:
categories_np = np.array(categories)
categories_class = np.argmax(categories_np, axis=1)
print(len(categories_class))
preds_np = np.array(preds)
preds_zero_one = np.array(preds_np > 0.5, dtype=np.int32)

categories_softmax = np.exp(categories_np) / np.sum(np.exp(categories_np), axis=1, keepdims=True)
print(len(categories_softmax))

category_probs = np.zeros((len(categories_softmax), 2))
category_count = [0, 0, 0, 0]
for row in range(len(category_probs)):
    cat_max_idx = np.argmax(categories_softmax[row])
    category_count[cat_max_idx] += 1
    prob = categories_softmax[row, cat_max_idx]
    
    print(f"cat_max: {cat_max_idx}, prob: {prob}")
    
    if      cat_max_idx == 0: category_probs[row] = np.array([1 - prob, 1 - prob])
    elif    cat_max_idx == 1: category_probs[row] = np.array([prob, 1 - prob])
    elif    cat_max_idx == 2: category_probs[row] = np.array([1 - prob, prob])
    elif    cat_max_idx == 3: category_probs[row] = np.array([prob, prob])
        
ensemble_probs = (category_probs + preds_np) / 2
print(category_count)

50000
50000
cat_max: 3, prob: 0.6024853202990836
cat_max: 2, prob: 0.8764756946835748
cat_max: 2, prob: 0.6273149169274469
cat_max: 2, prob: 0.891381661718048
cat_max: 2, prob: 0.8794715287548247
cat_max: 2, prob: 0.5732100542768371
cat_max: 2, prob: 0.8354402507488092
cat_max: 3, prob: 0.5481034195611262
cat_max: 2, prob: 0.7127185708766712
cat_max: 2, prob: 0.9086893838983574
cat_max: 2, prob: 0.8117316411084868
cat_max: 2, prob: 0.9263320445911939
cat_max: 2, prob: 0.8036134715272601
cat_max: 2, prob: 0.8725267598003266
cat_max: 2, prob: 0.8336814066601776
cat_max: 2, prob: 0.7752958774664132
cat_max: 2, prob: 0.7812474077371603
cat_max: 2, prob: 0.8996327722416927
cat_max: 2, prob: 0.7979860309462049
cat_max: 3, prob: 0.4970357979582642
cat_max: 2, prob: 0.9721327867815228
cat_max: 3, prob: 0.5315735580006438
cat_max: 2, prob: 0.9320010802471829
cat_max: 2, prob: 0.9481556274296593
cat_max: 2, prob: 0.7262385923191055
cat_max: 2, prob: 0.8533992047070608
cat_max: 3, prob: 0.6459532

In [102]:
submit = pd.read_csv('/home/work/joono/joono/dataset/sample_submission.csv')

max_thres = 0.99999
min_thres = 0.00001

fake, real = 0, 1

for i in tqdm(range(len(preds))):
    if      ensemble_probs[i][fake] > max_thres  : submit.iloc[i, 1] = 1
    elif    ensemble_probs[i][fake] < min_thres  : submit.iloc[i, 1] = 0 
    else                                         : submit.iloc[i, 1] = ensemble_probs[i][0]
    if      ensemble_probs[i][real] > max_thres  : submit.iloc[i, 2] = 1
    elif    ensemble_probs[i][real] < min_thres  : submit.iloc[i, 2] = 0 
    else                                         : submit.iloc[i, 2] = ensemble_probs[i][1]

  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipykernel_3700592/61870092.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6293802963342342' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                                         : submit.iloc[i, 1] = ensemble_probs[i][0]
/tmp/ipykernel_3700592/61870092.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.716889630421606' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else                                         : submit.iloc[i, 2] = ensemble_probs[i][1]
100%|██████████| 50000/50000 [00:08<00:00, 6077.43it/s]


In [103]:
submit.to_csv('AASIST_train_test_align_centerloss_ce_ensemble_test.csv', index=False)

In [ ]:
preds

In [ ]:
import os
import pathlib
from glob import glob

In [ ]:

ids, paths = [], []
for ul_path in glob("/home/work/joono/joono/dataset/unlabeled_data/*"):
    basename = os.path.basename(ul_path)
    id = basename.split(".")[0]
    path = os.path.join(".", "unlabeled_data", basename)
    
    ids.append(id)
    paths.append(path)

In [ ]:
df = pd.DataFrame({
    "id": ids,
    "path": paths
})

In [ ]:
df.to_csv("unlabled_data.csv", index=False)

In [ ]:
paths[763]